In [173]:
libraries = c("dplyr","magrittr","tidyr","ggplot2","rstan","readxl")
for(x in libraries) { library(x,character.only=TRUE,warn.conflicts=FALSE,quietly=TRUE) }

require(zoo)
require(lubridate)

base_sz = 12 # base_size parameter
theme_set(theme_bw())

'%&%' = function(x,y) paste0(x,y)

options(mc.cores = parallel::detectCores())
rstan_options(auto_write = TRUE)

packageVersion("rstan")
packageVersion("StanHeaders")
rstan::stan_version()

[1] ‘2.19.3’

[1] ‘2.21.0.1’

[1] "2.21.0"

In [189]:
datafilename = "../../../Hokkaido_Wuhan Data 2020/wuhan_data_master.xlsx"
read_excel(datafilename, sheet="export") -> Df
names(Df)

[1] "ID"                            "DeathID"                      
 [3] "Age"                           "AgeDecade"                    
 [5] "Sex"                           "InWuhan2WeeksBeforeOnset"     
 [7] "ExposureL"                     "ExposureR"                    
 [9] "ExposureType"                  "ClusterName"                  
[11] "Onset"                         "DateCaseFirstSoughtHealthcare"
[13] "DateHIQ"                       "HIQ"                          
[15] "DateOfDeath"                   "DateDischarged"               
[17] "DateReportedConfirmed"         "ResidenceCity"                
[19] "ResidenceProvince"             "OfficialResidenceCity"        
[21] "OfficialResidenceProvince"     "ResidenceCountry"             
[23] "Nationality"                   "DiagnosisCity"                
[25] "DiagnosisProvince"             "DiagnosisCountry"             
[27] "LocalCaseNum"                  "SIGroup"                      
[29] "IsInfectee"                    "InfectorID"                   
[31] "SIStatus"                      "SIType"                       
[33] "Asymptomatic"                  "Pneumonia"                    
[35] "WuhanFreshMarket"              "WuhanSCSCVisit"               
[37] "DateSpecColl"                  "LabPosDate"                   
[39] "CaseClass"                     "Evacuaee"                     
[41] "CruiseShip"                    "URL"                          
[43] "Note"

In [190]:
Df %>% filter((SIStatus=='Certain')|(SIStatus=='Probable')) %>% select(ID, SIStatus, Onset, ExposureL, DateReportedConfirmed, InfectorID) %>% 
    mutate(ID = as.character(ID), 
           InfectorID = as.character(InfectorID), 
           SIStatus = as.character(SIStatus), 
           OnsetL = if_else(!is.na(Onset), Onset, ExposureL), 
           OnsetR = if_else(!is.na(Onset), Onset+days(1), DateReportedConfirmed)) %>%
    rename(S_L = OnsetL, S_R = OnsetR) -> df

lvls <- levels(Df$ID)
df

ID,SIStatus,Onset,ExposureL,DateReportedConfirmed,InfectorID,S_L,S_R
<chr>,<chr>,<dttm>,<dttm>,<dttm>,<chr>,<dttm>,<dttm>
NW089,Certain,2020-01-20,2020-01-17,2020-01-23,NW088,2020-01-20,2020-01-21
NW327,Probable,2020-01-19,2020-01-17,2020-02-01,NW088,2020-01-19,2020-01-20
NW281,Certain,2020-01-26,2020-01-22,2020-01-30,NW160,2020-01-26,2020-01-27
NW190,Certain,2020-01-26,2020-01-20,2020-01-28,NW188,2020-01-26,2020-01-27
NW191,Certain,2020-01-26,2020-01-21,2020-01-28,NW188,2020-01-26,2020-01-27
NW200,Probable,2020-01-26,NA,2020-01-28,NW199,2020-01-26,2020-01-27
NW228,Probable,2020-01-24,2020-01-19,2020-01-29,NW241,2020-01-24,2020-01-25
NW229,Probable,2020-01-22,NA,2020-01-29,NW241,2020-01-22,2020-01-23
NW648,Probable,2020-02-03,NA,2020-02-08,NW262,2020-02-03,2020-02-04


In [191]:
Df %>% select(ID, Onset, ExposureL, DateReportedConfirmed) %>% 
    mutate(OnsetL = if_else(!is.na(Onset), Onset, ExposureL), 
           OnsetR = if_else(!is.na(Onset), Onset+days(1), DateReportedConfirmed)) %>%
    select(OnsetL, OnsetR, ID) %>% rename(E_L=OnsetL, E_R = OnsetR, InfectorID = ID) %>% 
    mutate(InfectorID = as.character(InfectorID)) %>%
    left_join(df %>% select(ID, InfectorID, SIStatus, S_L, S_R), by='InfectorID') %>%
     drop_na %>% mutate_each(as.Date, -ID, -InfectorID, -SIStatus) %>% select(ID, InfectorID, SIStatus, everything()) %>% arrange(SIStatus) -> df

df

ID,InfectorID,SIStatus,E_L,E_R,S_L,S_R
<chr>,<chr>,<chr>,<date>,<date>,<date>,<date>
NW285,NW281,Certain,2020-01-26,2020-01-27,2020-01-29,2020-01-30
NW286,NW281,Certain,2020-01-26,2020-01-27,2020-01-30,2020-01-31
NW544,NW281,Certain,2020-01-26,2020-01-27,2020-01-30,2020-01-31
NW089,NW088,Certain,2020-01-17,2020-01-18,2020-01-20,2020-01-21
NW190,NW188,Certain,2020-01-24,2020-01-25,2020-01-26,2020-01-27
NW191,NW188,Certain,2020-01-24,2020-01-25,2020-01-26,2020-01-27
NW667,NW666,Certain,2019-12-20,2019-12-21,2019-12-25,2019-12-26
NW668,NW666,Certain,2019-12-20,2019-12-21,2019-12-29,2019-12-30
NW670,NW669,Certain,2019-12-27,2019-12-28,2020-01-03,2020-01-04


In [193]:
CUTOFF_TIME = as.Date('2020-02-12')
t0 = as.Date('2019-12-01')

df['tstar'] = CUTOFF_TIME
df %<>% mutate_each(list(~(as.numeric(. - t0))), -ID, -InfectorID, -SIStatus) %>% mutate(dist = (S_R+S_L)/2-(E_R+E_L)/2)

df %<>% mutate(S_L = if_else(S_L < E_L, E_L, S_L), E_R = if_else(E_R > S_R, S_R, E_R)) 

df

ID,InfectorID,SIStatus,E_L,E_R,S_L,S_R,tstar,dist
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
NW285,NW281,Certain,56,57,59,60,73,3
NW286,NW281,Certain,56,57,60,61,73,4
NW544,NW281,Certain,56,57,60,61,73,4
NW089,NW088,Certain,47,48,50,51,73,3
NW190,NW188,Certain,54,55,56,57,73,2
NW191,NW188,Certain,54,55,56,57,73,2
NW667,NW666,Certain,19,20,24,25,73,5
NW668,NW666,Certain,19,20,28,29,73,9
NW670,NW669,Certain,26,27,33,34,73,7


In [195]:
mean(df$dist)

[1] 4.642857

In [196]:
sd(df$dist)

[1] 2.613558

In [197]:
data_drname = "../../data"
flname = 'data-probable.csv'
write.table(df, paste0(data_drname,flname), row.names=FALSE, sep=",", quote = FALSE)

# Stan simulations

In [198]:
stanmaindir = 'stan-sims-probable'
unlink(stanmaindir, recursive=T)
dir.create(stanmaindir)

# No truncation

## <font color="maroon">Lognormal distribution</font>

In [206]:
## main dir for Stan simulations
standirname = stanmaindir%&%"/lognormal-no_truncation"
unlink(standirname, recursive=T)
dir.create(standirname)

# Dumping data
N = nrow(df)
E_L = df$E_L
E_R = df$E_R
S_L = df$S_L
S_R = df$S_R
stan_rdump(c('E_L', 'E_R', 'S_L', 'S_R', 'N'), file=standirname%&%"/Data.R") 

# Dumping initial conditions
e_raw = rep(.5, N)
s_raw = rep(.5, N)
logmean_SI = log(mean(df$dist))
logsd_SI = log(sd(df$dist))
stan_rdump(c('e_raw', 's_raw', 'logmean_SI', 'logsd_SI'), file=standirname%&%"/Init.R") 

# Stan program
"data {
    int<lower = 0> N; // number of records
    vector<lower = 0>[N] E_L;
    vector<lower = 0>[N] E_R;
    vector<lower = 0>[N] S_L;
    vector<lower = 0>[N] S_R;
}

parameters {
    real logmean_SI;
    real logsd_SI;

    vector<lower = 0, upper = 1>[N] s_raw;
    vector<lower = 0, upper = 1>[N] e_raw;
}

transformed parameters {
    real<lower = 0> param2 = sqrt(log((exp(2*(logsd_SI-logmean_SI))+1.0)));
    real param1 = logmean_SI - param2^2/2.0;

    vector<lower = min(S_L), upper = max(S_R)>[N] s;
    vector<lower = min(E_L), upper = max(E_R)>[N] e;
    vector<lower = 0, upper = max(S_R)>[N] t;

    {
        vector[N] s_;

        
        s_ = S_L + (S_R - S_L) .* s_raw;
        for (k in 1:N) {
            if (s_[k] < E_L[k])
                s[k] = E_L[k];
            else 
                s[k] = s_[k];

            if (E_R[k] > s[k]) 
                e[k] = E_L[k] + (s[k] - 1e-3 - E_L[k]) * e_raw[k];
            else if (E_L[k] > s[k])
                e[k] = s[k] - 1e-3;
            else
                e[k] = E_L[k] + (E_R[k] - E_L[k]) * e_raw[k];
        }
        t = s - e;
    }
}

model {
    logmean_SI ~ std_normal();
    logsd_SI ~ std_normal();

    e_raw ~ normal(0.5, 1.0);
    s_raw ~ normal(0.5, 1.0);

    t ~ lognormal(param1, param2);
}

generated quantities {
    real<lower = 0> mean_SI = exp(param1 + param2^2/2);
    real<lower = 0> sd_SI = sqrt((exp(param2^2)-1)*exp(2*param1+param2^2));

    vector[N] log_likelihood;
    for (k in 1:N) 
        log_likelihood[k] = lognormal_lpdf(t[k] | param1, param2);
}" %>% cat(file=standirname %&% "/fit.stan", sep="", fill=TRUE)

standistribdir = "../../../CmdStan"
stanscriptdir = "../Hokkaido_Wuhan_Serial_Interval_2020/scripts/Andrei/"%&%standirname
## bash file
"#!/bin/bash
cwd=$(pwd)
cd "%&%standistribdir%&%"
make -j6 "%&%stanscriptdir%&%"/fit
cd "%&%stanscriptdir%&%"
mkdir -p diagnostics
for i in {1..10}
do
    echo Running ${i}
    SEEDNUMBER=$((1+$i))
    ./fit \\
        method=sample num_samples=10000 num_warmup=10000 save_warmup=0 \\
            adapt delta=0.92 \\
            algorithm=hmc \\
                engine=nuts \\
        random seed=${SEEDNUMBER} \\
        id=$i \\
        data file=Data.R \\
        init=Init.R \\
        output file=trace-$i.csv \\
            diagnostic_file=diagnostics/diagnostics-$i.csv > diagnostics/output-$i.txt &
done
echo Finished sampling haha!
" %>% cat(file=standirname%&%"/fit.sh", sep="", fill=TRUE)

## running the bash script
system("bash "%&%standirname%&%"/fit.sh", intern = TRUE)

[1] ""                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
 [2] "--- Translating Stan model to C++ code ---"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
 [3] "bin/stanc  --o=../Hokkaido_Wuhan_Serial_Interval_2020/scripts/Andrei/stan-sims-probable/lognormal-no_truncation/fit.hpp

## <font color="maroon">Gamma distribution</font>

In [207]:
## main dir for Stan simulations
standirname = stanmaindir%&%"/gamma-no_truncation"
unlink(standirname, recursive=T)
dir.create(standirname)

# Dumping data
N = nrow(df)
E_L = df$E_L
E_R = df$E_R
S_L = df$S_L
S_R = df$S_R
stan_rdump(c('E_L', 'E_R', 'S_L', 'S_R', 'N'), file=standirname%&%"/Data.R") 

# Dumping initial conditions
e_raw = rep(.2, N)
s_raw = rep(.8, N)
param1 = (mean(df$dist)/sd(df$dist))^2
param2 = mean(df$dist)/(sd(df$dist)^2)
stan_rdump(c('e_raw', 's_raw', 'param1', 'param2'), file=standirname%&%"/Init.R") 

# Stan program
"data {
    int<lower = 0> N; // number of records
    vector<lower = 0>[N] E_L;
    vector<lower = 0>[N] E_R;
    vector<lower = 0>[N] S_L;
    vector<lower = 0>[N] S_R;
}

parameters {
    real<lower=0> mean_SI;
    real<lower=0> sd_SI;

    vector<lower = 0, upper = 1>[N] e_raw;
    vector<lower = 0, upper = 1>[N] s_raw;
}

transformed parameters {
    real<lower = 0> param1 = (mean_SI/sd_SI)^2;
    real<lower = 0> param2 = mean_SI/(sd_SI^2);

    vector<lower = min(S_L), upper = max(S_R)>[N] s;
    vector<lower = min(E_L), upper = max(E_R)>[N] e;
    vector<lower = 0, upper = max(S_R)>[N] t;

    {
        vector[N] s_;
        
        s_ = S_L + (S_R - S_L) .* s_raw;
        for (k in 1:N) {
            if (s_[k] < E_L[k])
                s[k] = E_L[k];
            else 
                s[k] = s_[k];

            if (E_R[k] > s[k]) 
                e[k] = E_L[k] + (s[k] - 1e-3 - E_L[k]) * e_raw[k];
            else if (E_L[k] > s[k])
                e[k] = s[k] - 1e-3;
            else
                e[k] = E_L[k] + (E_R[k] - E_L[k]) * e_raw[k];
        }
        t = s - e;
    }
}

model {
    mean_SI ~ normal(5.0, 10.0);
    sd_SI ~ cauchy(0, 5.0);

    e_raw ~ normal(0.5, 1.0);
    s_raw ~ normal(0.5, 1.0);

    t ~ gamma(param1, param2);
}

generated quantities {
    vector[N] log_likelihood;
    for (k in 1:N) 
        log_likelihood[k] = gamma_lpdf(t[k] | param1, param2);
}" %>% cat(file=standirname %&% "/fit.stan", sep="", fill=TRUE)

standistribdir = "../../../CmdStan"
stanscriptdir = "../Hokkaido_Wuhan_Serial_Interval_2020/scripts/Andrei/"%&%standirname
## bash file
"#!/bin/bash
cwd=$(pwd)
cd "%&%standistribdir%&%"
make -j6 "%&%stanscriptdir%&%"/fit
cd "%&%stanscriptdir%&%"
mkdir -p diagnostics
for i in {1..10}
do
    echo Running ${i}
    SEEDNUMBER=$((1+$i))
    ./fit \\
        method=sample num_samples=10000 num_warmup=10000 save_warmup=0 \\
            adapt delta=0.92 \\
            algorithm=hmc \\
                engine=nuts \\
        random seed=${SEEDNUMBER} \\
        id=$i \\
        data file=Data.R \\
        init=Init.R \\
        output file=trace-$i.csv \\
            refresh=1000 \\
            diagnostic_file=diagnostics/diagnostics-$i.csv > diagnostics/output-$i.txt &
done
echo Finished sampling haha!
" %>% cat(file=standirname%&%"/fit.sh", sep="", fill=TRUE)

## running the bash script
system("bash "%&%standirname%&%"/fit.sh", intern = TRUE)

[1] ""                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
 [2] "--- Translating Stan model to C++ code ---"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
 [3] "bin/stanc  --o=../Hokkaido_Wuhan_Serial_Interval_2020/scripts/Andrei/stan-sims-probable/gamma-no_truncation/fit.hpp ../Hokkaido_Wuhan_S

## <font color="maroon">Weibull distribution</font>

In [208]:
## main dir for Stan simulations
standirname = stanmaindir%&%"/weibull-no_truncation"
unlink(standirname, recursive=T)
dir.create(standirname)

# Dumping data
N = nrow(df)
E_L = df$E_L
E_R = df$E_R
S_L = df$S_L
S_R = df$S_R
stan_rdump(c('E_L', 'E_R', 'S_L', 'S_R', 'N'), file=standirname%&%"/Data.R") 

# Dumping initial conditions
e_raw = rep(.2, N)
s_raw = rep(.8, N)
logmean_SI = log(mean(df$dist))
param1 = 1.75
stan_rdump(c('e_raw', 's_raw', 'logmean_SI', 'param1'), file=standirname%&%"/Init.R") 

# Stan program
"data {
    int<lower = 0> N; // number of records
    vector<lower = 0>[N] E_L;
    vector<lower = 0>[N] E_R;
    vector<lower = 0>[N] S_L;
    vector<lower = 0>[N] S_R;
}

parameters {
    real<lower = 0> mean_SI;
    real<lower = 0> param1;

    vector<lower = 0, upper = 1>[N] e_raw;
    vector<lower = 0, upper = 1>[N] s_raw;
}

transformed parameters {
    real<lower = 0> param2 = mean_SI/tgamma(1.0+1.0/param1);

    vector<lower = min(S_L), upper = max(S_R)>[N] s;
    vector<lower = min(E_L), upper = max(E_R)>[N] e;
    vector<lower = 0, upper = max(S_R)>[N] t;

    {
        vector[N] s_;
        
        s_ = S_L + (S_R - S_L) .* s_raw;
        for (k in 1:N) {
            if (s_[k] < E_L[k])
                s[k] = E_L[k];
            else 
                s[k] = s_[k];

            if (E_R[k] > s[k]) 
                e[k] = E_L[k] + (s[k] - 1e-3 - E_L[k]) * e_raw[k];
            else if (E_L[k] > s[k])
                e[k] = s[k] - 1e-3;
            else
                e[k] = E_L[k] + (E_R[k] - E_L[k]) * e_raw[k];
        }
        t = s - e;
    }
}

model {
    mean_SI ~ normal(5.0, 10.0);
    param1 ~ exponential(0.0001);

    e_raw ~ normal(0.5, 1.0);
    s_raw ~ normal(0.5, 1.0);

    t ~ weibull(param1, param2);
}

generated quantities {
    real sd_SI = param2*sqrt(tgamma(1.0+2.0/param1)-(tgamma(1.0+1.0/param1))^2);

    vector[N] log_likelihood;
    for (k in 1:N) 
        log_likelihood[k] = weibull_lpdf(t[k] | param1, param2);
}" %>% cat(file=standirname %&% "/fit.stan", sep="", fill=TRUE)

standistribdir = "../../../CmdStan"
stanscriptdir = "../Hokkaido_Wuhan_Serial_Interval_2020/scripts/Andrei/"%&%standirname
## bash file
"#!/bin/bash
cwd=$(pwd)
cd "%&%standistribdir%&%"
make -j6 "%&%stanscriptdir%&%"/fit
cd "%&%stanscriptdir%&%"
mkdir -p diagnostics
for i in {1..10}
do
    echo Running ${i}
    SEEDNUMBER=$((1+$i))
    ./fit \\
        method=sample num_samples=10000 num_warmup=10000 save_warmup=0 \\
            adapt delta=0.92 \\
            algorithm=hmc \\
                engine=nuts \\
        random seed=${SEEDNUMBER} \\
        id=$i \\
        data file=Data.R \\
        init=Init.R \\
        output file=trace-$i.csv \\
            diagnostic_file=diagnostics/diagnostics-$i.csv > diagnostics/output-$i.txt &
done
echo Finished sampling haha!
" %>% cat(file=standirname%&%"/fit.sh", sep="", fill=TRUE)

## running the bash script
system("bash "%&%standirname%&%"/fit.sh", intern = TRUE)

[1] ""                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
 [2] "--- Translating Stan model to C++ code ---"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
 [3] "bin/stanc  --o=../Hokkaido_Wuhan_Serial_Interval_2020/scripts/Andrei/stan-sims-probable/weibull-no_truncation/fit.hpp ../Hokkai

# With truncation

## <font color="red">Lognormal distribution</font>

In [211]:
## main dir for Stan simulations
standirname = stanmaindir%&%"/lognormal-truncated"
unlink(standirname, recursive=T)
dir.create(standirname)

# Dumping data
N = nrow(df)
E_L = df$E_L
E_R = df$E_R
S_L = df$S_L
S_R = df$S_R
r = 0.14
upper_bound = df$tstar[1]
stan_rdump(c('E_L', 'E_R', 'S_L', 'S_R', 'N', 'r', 'upper_bound'), file=standirname%&%"/Data.R") 

# Dumping initial conditions
e_raw = rep(.2, N)
s_raw = rep(.8, N)
logmean_SI = log(mean(df$dist))
logsd_SI = log(sd(df$dist))
stan_rdump(c('e_raw', 's_raw', 'logmean_SI', 'logsd_SI'), file=standirname%&%"/Init.R") 

# Stan program
"functions {
    real[] fstar_ode(real t, real[] z, real[] theta, data real[] x_r, int[] x_i) {
        int N = x_i[1]; // number of records

        real e[N] = theta[1:N];
        real param1 = theta[N+1];
        real param2 = theta[N+2];

        real upper_bound = x_r[1];
        real r = x_r[2];

        real dzdt[N];
        real tstar[N];

        for (k in 1:N) {
            tstar[k] = upper_bound - e[k];
            dzdt[k] = exp(lognormal_lcdf(tstar[k]*(1.0-t) | param1, param2)) * r * tstar[k] * exp(-r*tstar[k]*t) / (1.0 - exp(-r*tstar[k]*t));
        }

        return dzdt;
    }
}

data {
    int<lower = 0> N; // number of records
    vector<lower = 0>[N] E_L;
    vector<lower = 0>[N] E_R;
    vector<lower = 0>[N] S_L;
    vector<lower = 0>[N] S_R;
    real<lower = 0> upper_bound;
    real<lower = 0> r;
}

transformed data {
    int X_i[1] = {N};

    real X_r[2] = {upper_bound, r};
}

parameters {
    real logmean_SI;
    real logsd_SI;

    vector<lower = 0, upper = 1>[N] e_raw;
    vector<lower = 0, upper = 1>[N] s_raw;
}

transformed parameters {
    real<lower = 0> param2 = sqrt(log((exp(2*(logsd_SI-logmean_SI))+1.0)));
    real param1 = logmean_SI - param2^2/2.0;

    vector<lower = min(E_L), upper = max(E_R)>[N] e;
    vector<lower = min(S_L), upper = max(S_R)>[N] s;

    real Z[N]; 
    
    {
        real theta[N+2];
        real Z0[N];
        
        s = S_L + (S_R - S_L) .* s_raw;
        for (k in 1:N) 
            if (E_R[k] > s[k]) 
                e[k] = E_L[k] + (s[k] - E_L[k]) * e_raw[k];
            else
                e[k] = E_L[k] + (E_R[k] - E_L[k]) * e_raw[k];

        for (k in 1:N) {
            Z0[k] = 0.0;
            theta[k] = e[k];
        }
        theta[N+1] = param1;
        theta[N+2] = param2;

        Z = to_array_1d(integrate_ode_rk45(fstar_ode, Z0, 0.001, {1.0}, theta, X_r, X_i, 1e-5, 1e-3, 5e2));
    }
}

model {
    logmean_SI ~ std_normal();
    logsd_SI ~ std_normal();

    e_raw ~ normal(0.5, 1.0);
    s_raw ~ normal(0.5, 1.0);

    for (k in 1:N) 
        target += lognormal_lpdf(s[k] - e[k] | param1, param2) - log(Z[k]);
}

generated quantities {
    real<lower = 0> mean_SI = exp(param1 + param2^2/2);
    real<lower = 0> sd_SI = sqrt((exp(param2^2)-1)*exp(2*param1+param2^2));

    vector[N] log_likelihood;
    for (k in 1:N)
        log_likelihood[k] = lognormal_lpdf(s[k] - e[k] | param1, param2) - log(Z[k]);
}" %>% cat(file=standirname %&% "/fit.stan", sep="", fill=TRUE)

standistribdir = "../../../CmdStan"
stanscriptdir = "../Hokkaido_Wuhan_Serial_Interval_2020/scripts/Andrei/"%&%standirname
## bash file
"#!/bin/bash
cwd=$(pwd)
cd "%&%standistribdir%&%"
make -j6 "%&%stanscriptdir%&%"/fit
cd "%&%stanscriptdir%&%"
mkdir -p diagnostics
for i in {1..10}
do
    echo Running ${i}
    SEEDNUMBER=$((1+$i))
    ./fit \\
        method=sample num_samples=10000 num_warmup=10000 save_warmup=0 \\
            adapt delta=0.98 \\
            algorithm=hmc \\
                engine=nuts \\
        random seed=${SEEDNUMBER} \\
        id=$i \\
        data file=Data.R \\
        init=Init.R \\
        output file=trace-$i.csv \\
            diagnostic_file=diagnostics/diagnostics-$i.csv > diagnostics/output-$i.txt &
done
echo Finished sampling haha!
" %>% cat(file=standirname%&%"/fit.sh", sep="", fill=TRUE)

## running the bash script
system("bash "%&%standirname%&%"/fit.sh", intern = TRUE)

[1] ""                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
 [2] "--- Translating Stan model to C++ code ---"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
 [3] "bin/stanc  --o=../Hokkaido_Wuhan_Serial_Interval_2020/scripts/Andrei/stan-sims-probable/lognormal-truncated_new/fit.hpp

## <font color="red">Gamma distribution</font>

In [204]:
## main dir for Stan simulations
standirname = stanmaindir%&%"/gamma-truncated"
unlink(standirname, recursive=T)
dir.create(standirname)

# Dumping data
N = nrow(df)
E_L = df$E_L
E_R = df$E_R
S_L = df$S_L
S_R = df$S_R
r = 0.14
upper_bound = df$tstar[1]
stan_rdump(c('E_L', 'E_R', 'S_L', 'S_R', 'N', 'r', 'upper_bound'), file=standirname%&%"/Data.R") 

# Dumping initial conditions
e_raw = rep(.2, N)
s_raw = rep(.8, N)
mean_SI = mean(df$dist)
sd_SI = sd(df$dist)
stan_rdump(c('e_raw', 's_raw', 'mean_SI', 'sd_SI'), file=standirname%&%"/Init.R") 

# Stan program
"functions {
    real[] fstar_ode(real t, real[] z, real[] theta, data real[] x_r, int[] x_i) {
        int N = x_i[1]; // number of records

        real e[N] = theta[1:N];
        real param1 = theta[N+1];
        real param2 = theta[N+2];

        real upper_bound = x_r[1];
        real r = x_r[2];

        real dzdt[N];
        real tstar[N];

        for (k in 1:N) {
            tstar[k] = upper_bound - e[k];
            dzdt[k] = exp(gamma_lcdf(tstar[k]*(1.0-t) | param1, param2)) * r * tstar[k] * exp(-r*tstar[k]*t) / (1.0 - exp(-r*tstar[k]*t));
        }

        return dzdt;
    }
}

data {
    int<lower = 0> N; // number of records
    vector<lower = 0>[N] E_L;
    vector<lower = 0>[N] E_R;
    vector<lower = 0>[N] S_L;
    vector<lower = 0>[N] S_R;
    real<lower = 0> r;
    real<lower = 0> upper_bound;
}

transformed data {
    int X_i[1] = {N};

    real X_r[2] = {upper_bound, r};
}

parameters {
    real<lower = 0> mean_SI;
    real<lower = 0> sd_SI;

    vector<lower = 0, upper = 1>[N] s_raw;
    vector<lower = 0, upper = 1>[N] e_raw;
}

transformed parameters {
    real<lower = 0> param1 = (mean_SI/sd_SI)^2;
    real<lower = 0> param2 = mean_SI/(sd_SI^2);

    vector<lower = min(S_L), upper = max(S_R)>[N] s;
    vector<lower = min(E_L), upper = max(E_R)>[N] e;
    vector<lower = 0, upper = max(S_R)>[N] t;

    real Z[N]; 
    
    {
        vector[N] s_;
        real theta[N+2];
        real Z0[N];
        
        s_ = S_L + (S_R - S_L) .* s_raw;
        for (k in 1:N) {
            if (s_[k] < E_L[k])
                s[k] = E_L[k];
            else 
                s[k] = s_[k];

            if (E_R[k] > s[k]) 
                e[k] = E_L[k] + (s[k] - 1e-3 - E_L[k]) * e_raw[k];
            else if (E_L[k] > s[k])
                e[k] = s[k] - 1e-3;
            else
                e[k] = E_L[k] + (E_R[k] - E_L[k]) * e_raw[k];
        }
        t = s - e;

        for (k in 1:N) {
            Z0[k] = 0.0;
            theta[k] = s[k] - t[k];
        }
        theta[N+1] = param1;
        theta[N+2] = param2;

        Z = to_array_1d(integrate_ode_rk45(fstar_ode, Z0, 0.001, {1.0}, theta, X_r, X_i, 1e-5, 1e-3, 5e2));
    }
}

model {
    mean_SI ~ normal(5.0, 10.0);
    sd_SI ~ cauchy(0, 5.0);

    e_raw ~ normal(0.5, 1.0);
    s_raw ~ normal(0.5, 1.0);

    for (k in 1:N) 
        target += gamma_lpdf(t[k] | param1, param2) - log(Z[k]);
}

generated quantities {
    vector[N] log_likelihood;
    for (k in 1:N) 
        log_likelihood[k] = gamma_lpdf(t[k] | param1, param2) - log(Z[k]);
}" %>% cat(file=standirname %&% "/fit.stan", sep="", fill=TRUE)

standistribdir = "../../../CmdStan"
stanscriptdir = "../Hokkaido_Wuhan_Serial_Interval_2020/scripts/Andrei/"%&%standirname
## bash file
"#!/bin/bash
cwd=$(pwd)
cd "%&%standistribdir%&%"
make -j6 "%&%stanscriptdir%&%"/fit
cd "%&%stanscriptdir%&%"
mkdir -p diagnostics
for i in {1..10}
do
    echo Running ${i}
    SEEDNUMBER=$((1+$i))
    ./fit \\
        method=sample num_samples=10000 num_warmup=10000 save_warmup=0 \\
            adapt delta=0.98 \\
            algorithm=hmc \\
                engine=nuts \\
        random seed=${SEEDNUMBER} \\
        id=$i \\
        data file=Data.R \\
        init=Init.R \\
        output file=trace-$i.csv \\
            diagnostic_file=diagnostics/diagnostics-$i.csv > diagnostics/output-$i.txt &
done
echo Finished sampling haha!
" %>% cat(file=standirname%&%"/fit.sh", sep="", fill=TRUE)

## running the bash script
system("bash "%&%standirname%&%"/fit.sh", intern = TRUE)

[1] ""                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
 [2] "--- Translating Stan model to C++ code ---"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
 [3] "bin/stanc  --o=../Hokkaido_Wuhan_Serial_Interval_2020/scripts/Andrei/stan-sims-probable/gamma-truncated/fit.hpp ../Hokkaido_Wuhan_Serial_Interval_2020/

## <font color="red">Weibull distribution</font>

In [209]:
## main dir for Stan simulations
standirname = stanmaindir%&%"/weibull-truncated"
unlink(standirname, recursive=T)
dir.create(standirname)

# Dumping data
N = nrow(df)
E_L = df$E_L
E_R = df$E_R
S_L = df$S_L
S_R = df$S_R
r = 0.14
upper_bound = df$tstar[1]
stan_rdump(c('E_L', 'E_R', 'S_L', 'S_R', 'N', 'r', 'upper_bound'), file=standirname%&%"/Data.R") 

# Dumping initial conditions
e_raw = rep(.2, N)
s_raw = rep(.8, N)
logmean_SI = log(mean(df$dist))
param1 = 1.75
stan_rdump(c('e_raw', 's_raw', 'logmean_SI', 'param1'), file=standirname%&%"/Init.R") 

# Stan program
"functions {
    real[] fstar_ode(real t, real[] z, real[] theta, data real[] x_r, int[] x_i) {
        int N = x_i[1]; // number of records

        real e[N] = theta[1:N];
        real param1 = theta[N+1];
        real param2 = theta[N+2];

        real upper_bound = x_r[1];
        real r = x_r[2];

        real dzdt[N];
        real tstar[N];

        for (k in 1:N) {
            tstar[k] = upper_bound - e[k];
            dzdt[k] = - expm1(-((1.0-t)*tstar[k]/param2)^param1) * r * tstar[k] * exp(-r*tstar[k]*t) / (1.0 - exp(-r*tstar[k]*t));
        }

        return dzdt;
    }
}

data {
    int<lower = 0> N; // number of records
    vector<lower = 0>[N] E_L;
    vector<lower = 0>[N] E_R;
    vector<lower = 0>[N] S_L;
    vector<lower = 0>[N] S_R;
    real<lower = 0> r;
    real<lower = 0> upper_bound;
}

transformed data {
    int X_i[1] = {N};

    real X_r[2] = {upper_bound, r};
}

parameters {
    real<lower = 0> mean_SI;
    real<lower = 0> param1;

    vector<lower = 0, upper = 1>[N] e_raw;
    vector<lower = 0, upper = 1>[N] s_raw;
}

transformed parameters {
    real<lower = 0> param2 = mean_SI/tgamma(1.0+1.0/param1);

    vector<lower = min(S_L), upper = max(S_R)>[N] s;
    vector<lower = min(E_L), upper = max(E_R)>[N] e;
    vector<lower = 0, upper = max(S_R)>[N] t;

    real Z[N]; 
    
    {
        vector[N] s_;
        real theta[N+2];
        real Z0[N];
        
        s_ = S_L + (S_R - S_L) .* s_raw;
        for (k in 1:N) {
            if (s_[k] < E_L[k])
                s[k] = E_L[k];
            else 
                s[k] = s_[k];

            if (E_R[k] > s[k]) 
                e[k] = E_L[k] + (s[k] - 1e-3 - E_L[k]) * e_raw[k];
            else if (E_L[k] > s[k])
                e[k] = s[k] - 1e-3;
            else
                e[k] = E_L[k] + (E_R[k] - E_L[k]) * e_raw[k];
        }
        t = s - e;

        for (k in 1:N) {
            Z0[k] = 0.0;
            theta[k] = s[k] - t[k];
        }
        theta[N+1] = param1;
        theta[N+2] = param2;

        Z = to_array_1d(integrate_ode_rk45(fstar_ode, Z0, 0.001, {1.0}, theta, X_r, X_i, 1e-5, 1e-3, 5e2));
    }
}

model {
    mean_SI ~ normal(5.0, 10.0);
    param1 ~ exponential(0.0001);

    e_raw ~ normal(0.5, 1.0);
    s_raw ~ normal(0.5, 1.0);

    for (k in 1:N) 
        target += weibull_lpdf(t[k] | param1, param2) - log(Z[k]);
}

generated quantities {
    real sd_SI = param2*sqrt(tgamma(1.0+2.0/param1)-(tgamma(1.0+1.0/param1))^2);

    vector[N] log_likelihood;
    for (k in 1:N) 
        log_likelihood[k] = weibull_lpdf(t[k] | param1, param2) - log(Z[k]);
}" %>% cat(file=standirname %&% "/fit.stan", sep="", fill=TRUE)

standistribdir = "../../../CmdStan"
stanscriptdir = "../Hokkaido_Wuhan_Serial_Interval_2020/scripts/Andrei/"%&%standirname
## bash file
"#!/bin/bash
cwd=$(pwd)
cd "%&%standistribdir%&%"
make -j6 "%&%stanscriptdir%&%"/fit
cd "%&%stanscriptdir%&%"
mkdir -p diagnostics
for i in {1..10}
do
    echo Running ${i}
    SEEDNUMBER=$((1+$i))
    ./fit \\
        method=sample num_samples=10000 num_warmup=10000 save_warmup=0 \\
            adapt delta=0.98 \\
            algorithm=hmc \\
                engine=nuts \\
        random seed=${SEEDNUMBER} \\
        id=$i \\
        data file=Data.R \\
        init=Init.R \\
        output file=trace-$i.csv \\
            diagnostic_file=diagnostics/diagnostics-$i.csv > diagnostics/output-$i.txt &
done
echo Finished sampling haha!
" %>% cat(file=standirname%&%"/fit.sh", sep="", fill=TRUE)

## running the bash script
system("bash "%&%standirname%&%"/fit.sh", intern = TRUE)

[1] ""                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
 [2] "--- Translating Stan model to C++ code ---"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
 [3] "bin/stanc  --o=../Hokkaido_Wuhan_Serial_Interval_2020/scripts/Andrei/stan-sims-probable/weibull-truncated/fit.hpp ../Hokkaido_Wuhan_Serial_Inte